In [55]:
import os
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gpd
import h5pyd

In [28]:
dir = "/home/rodr/buffalo/rodr/temp/"
files = os.listdir(dir)
ds = xr.open_dataset(dir + files[0])
ds

<xarray.Dataset> Size: 7TB
Dimensions:                   (phony_dim_0: 17520, phony_dim_1: 2018267)
Dimensions without coordinates: phony_dim_0, phony_dim_1
Data variables: (12/27)
    air_temperature           (phony_dim_0, phony_dim_1) float64 283GB ...
    alpha                     (phony_dim_0, phony_dim_1) float64 283GB ...
    aod                       (phony_dim_0, phony_dim_1) float64 283GB ...
    asymmetry                 (phony_dim_0, phony_dim_1) float64 283GB ...
    cld_opd_dcomp             (phony_dim_0, phony_dim_1) float64 283GB ...
    cld_press_acha            (phony_dim_0, phony_dim_1) float64 283GB ...
    ...                        ...
    surface_albedo            (phony_dim_0, phony_dim_1) float64 283GB ...
    surface_pressure          (phony_dim_0, phony_dim_1) float64 283GB ...
    time_index                (phony_dim_0) <U25 2MB ...
    total_precipitable_water  (phony_dim_0, phony_dim_1) float64 283GB ...
    wind_direction            (phony_dim_0, phony_dim_1) float64 283GB ...
    wind_speed                (phony_dim_0, phony_dim_1) float64 283GB ...
Attributes:
    full_version_record:  {"rex": "0.2.80", "pandas": "2.0.0", "numpy": "1.23...
    package:              rex
    version:              4.0.0

In [ ]:
vars = ["ghi", "dni", "relative_humidity", "air_temperature", "wind_speed"]
ds = xr.open_dataset(dir + files[0]).rename_vars(
    {"time_index": "time"}).set_coords("time").swap_dims(
    {"phony_dim_0": "time"}).rename_dims({"phony_dim_1": "points"})[vars]
ds["time"] = ds["time"].astype(np.datetime64)
ds = ds.sel({"time": slice(None, None, 2),
    "points": points.sort_index().index})
ds.nbytes/1024**3/4

/tmp/ipykernel_147351/947850732.py:5: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  ds["time"] = ds["time"].astype(np.datetime64)


329.31604743003845

In [51]:
ds["time"]

<xarray.DataArray 'time' (time: 17520)> Size: 140kB
array(['2022-01-01T00:00:00.000000000', '2022-01-01T00:30:00.000000000',
       '2022-01-01T01:00:00.000000000', ..., '2022-12-31T22:30:00.000000000',
       '2022-12-31T23:00:00.000000000', '2022-12-31T23:30:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 140kB 2022-01-01 ... 2022-12-31T23:30:00

In [53]:
# Descargamos malla con HSDS
with h5pyd.File("/nrel/nsrdb/conus/nsrdb_conus_2022.h5") as f:
    meta = pd.DataFrame(f['meta'][...])
points = meta[ (meta["country"]==b"Mexico")
    | (meta["state"].isin([b"Texas", b"California", b"Nevada"]))
    ].sort_values(["latitude", "longitude"])

points

Descargando malla de CONUS


,latitude,longitude,elevation,timezone,country,state,county,gid_full
1882233,14.54,-92.230003,9,-6.0,b'Mexico',b'Chiapas',b'Suchiate',3212614
1881063,14.56,-92.250000,10,-6.0,b'Mexico',b'Chiapas',b'Suchiate',3210975
1882232,14.56,-92.230003,9,-6.0,b'Mexico',b'Chiapas',b'Suchiate',3212613
1883401,14.56,-92.209999,12,-6.0,b'Mexico',b'Chiapas',b'Suchiate',3214253
1879893,14.58,-92.269997,12,-6.0,b'Mexico',b'Chiapas',b'Suchiate',3209326
...,...,...,...,...,...,...,...,...
261483,42.00,-117.010002,1593,-7.0,b'United States',b'Nevada',b'Elko',915089
262343,42.00,-116.989998,1574,-7.0,b'United States',b'Nevada',b'Elko',916499
263204,42.00,-116.970001,1561,-7.0,b'United States',b'Nevada',b'Elko',917909
264065,42.00,-116.949997,1548,-7.0,b'United States',b'Nevada',b'Elko',919319


In [ ]:
# Regiones eléctricas
dir_d = "/home/rodr/Datos/recurso_solar/"
dir_r = "/datos/rodr/temp/recurso_solar/duck_curve/"
regiones_p = f"{dir_d}Mapas/Electric_regions"
regiones = gpd.read_file(regiones_p)
# Vemos en qué región cae cada punto
print("Determinando región de cada punto")
points["geometry"] = gpd.points_from_xy(
    points["longitude"], points["latitude"])
points = gpd.GeoDataFrame(points, crs = 4326)
points["REGION"] = np.nan
for row in regiones.itertuples():
    points["MASK_REG"] = points.within(row.geometry)
    points["REGION"] = points["REGION"].where(
        ~points["MASK_REG"], row.ID_NUMBER )
points = points.dropna()

points

Determinando región de cada punto
